# **Classification : 2 Class**


## **1.환경준비**

### (1) 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

### (2) 필요 함수 생성

* 딥러닝을 위한 데이터로더 만들기

In [ ]:
def make_DataSet(x_train, x_val, y_train, y_val, batch_size = 32) :

    # 데이터 텐서로 변환
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

    # TensorDataset 생성 : 텐서 데이터셋으로 합치기
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

    return train_loader, x_val_tensor, y_val_tensor

* 학습을 위한 함수

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)                  # 전체 데이터셋의 크기
    num_batches = len(dataloader)                   # 배치 크기
    tr_loss = 0
    model.train()                                   # 훈련 모드로 설정(드롭아웃 및 배치 정규화와 같은 계층을 훈련 모드로 변경)
    for batch, (X, y) in enumerate(dataloader):     # batch : 현재 배치 번호, (X, y) : 입력 데이터와 레이블
        X, y = X.to(device), y.to(device)           # X.to(device), y.to(device): 입력 데이터와 레이블을 지정된 장치(device, CPU 또는 GPU)로 이동

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        tr_loss += loss

        # Backpropagation
        loss.backward()             # 역전파를 통해 모델의 각 파라미터에 대한 손실의 기울기를 계산
        optimizer.step()            # 옵티마이저가 계산된 기울기를 사용하여 모델의 파라미터를 업데이트
        optimizer.zero_grad()       # 옵티마이저의 기울기 값 초기화. 기울기가 누적되는 것 방지

    tr_loss /= num_batches          # 모든 배치에서의 loss 평균

    return tr_loss.item()

* 검증을 위한 함수

In [ ]:
def evaluate(x_val_tensor, y_val_tensor, model, loss_fn, device):
    model.eval()                        # 모델을 평가 모드로 설정

    with torch.no_grad():               # 평가 과정에서 기울기를 계산하지 않도록 설정(메모리 사용을 줄이고 평가 속도를 높입니다.)
        x, y = x_val_tensor.to(device), y_val_tensor.to(device)
        pred = model(x)
        eval_loss = loss_fn(pred, y).item()    # 예측 값 pred와 실제 값 y 사이의 손실 계산

    return eval_loss, pred

* 학습곡선

In [ ]:
def dl_learning_curve(tr_loss_list, val_loss_list):

    epochs = list(range(1, len(tr_loss_list)+1))
    plt.plot(epochs, tr_loss_list, label='train_err', marker = '.')
    plt.plot(epochs, val_loss_list, label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (3) device 준비(cpu or gpu)

In [ ]:
# cpu 혹은 gpu 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

### (4) 데이터로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.3.csv"
data = pd.read_csv(path)
data.drop(['Age_scale1', 'AgeGroup', 'SibSp','Parch' ], axis = 1, inplace = True)
data.head()

## **2.데이터 준비**

Sex, Age, Fare 만 이용하여 Survived 를 예측하는 모델을 만들어 봅시다.

### (1) 데이터 준비

In [ ]:
target = 'Survived'
features = ['Sex', 'Age', 'Fare']
x = data.loc[:, features]
y = data.loc[:, target]

### (2) 가변수화

In [ ]:
x = pd.get_dummies(x, columns = ['Sex'], drop_first = True)
x.head()

### (3) 데이터분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state = 20)

### (4) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## **3.모델링1**

### (1) 딥러닝을 위한 준비작업

* make_DataLoader

In [ ]:
train_loader, x_val_ts, y_val_ts = make_DataSet(x_train, x_val, y_train, y_val, 32)

In [ ]:
# 첫번째 배치만 로딩해서 살펴보기
for x, y in train_loader:
    print(f"Shape of x [rows, columns]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

### (2) 모델 선언

In [ ]:
n_feature = x.shape[1]

# 모델 구조 설계
model = nn.Sequential(
            nn.Linear(n_feature, 1),
            nn.Sigmoid()                # [회귀와 다른점] 시그모이드 활성화 함수 추가
        ).to(device)

print(model)

* Loss function과 Optimizer

In [ ]:
loss_fn = nn.BCELoss()          # [회귀와 다른점] Bineary Cross Entropy
optimizer = Adam(model.parameters(), lr=0.01)

### (3) 학습

In [ ]:
epochs = 50
tr_loss_list, val_loss_list = [], []

for t in range(epochs):
    tr_loss = train(train_loader, model, loss_fn, optimizer, device)
    val_loss,_ = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 학습된 파라미터 확인

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name}, Value: {param.data}")

* 학습 곡선

In [ ]:
dl_learning_curve(tr_loss_list, val_loss_list)

### (4) 모델 평가

In [ ]:
_, pred = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)

* **[회귀와 다른점]** 예측결과는 0 ~ 1 사이의 확률값
    * 확률값을 0, 1로 변환 필요
    * np.where(조건문, True일때 값, False일때 값)

In [ ]:
pred.numpy()[:5]

In [ ]:
pred = np.where(pred.numpy() > .5, 1, 0)
pred[:5]

* confusion matrix

In [ ]:
confusion_matrix(y_val_ts.numpy(), pred)

* classification_report

In [ ]:
print(classification_report(y_val_ts.numpy(), pred))

## **4.딥러닝2 : 전체 feature**
* 이제 전체 데이터를 가지고 모델링을 시도해 보겠습니다.


### (1) 데이터 전처리

* 데이터 준비

In [ ]:
target = 'Survived'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 가변수화

In [ ]:
cat_cols = ['Pclass','Sex', 'Embarked']
x = pd.get_dummies(x, columns = cat_cols, drop_first = True)

* 데이터분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state = 20)

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (2) 모델링

#### 1) 딥러닝을 위한 준비작업

* make_DataLoader

In [ ]:
train_loader, x_val_ts, y_val_ts = make_DataSet(x_train, x_val, y_train, y_val, 32)

In [ ]:
# 첫번째 배치만 로딩해서 살펴보기
for x, y in train_loader:
    print(f"Shape of x [rows, columns]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

#### 2) 모델 선언

In [ ]:
n_feature = x.shape[1]

# 모델 구조 설계
model = nn.Sequential(
                ,
                                # 시그모이드 활성화 함수 추가
        ).to(device)

print(model)

* Loss function과 Optimizer

In [ ]:
loss_fn =            # Bineary Cross Entropy
optimizer =

#### 3) 학습

In [ ]:
epochs = 50
tr_loss_list, val_loss_list = [], []

for t in range( ):
    tr_loss =
    val_loss,_ =

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 학습된 파라미터 확인

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name}, Value: {param.data}")

* 학습 곡선

In [ ]:
dl_learning_curve(tr_loss_list, val_loss_list)

#### 4) 모델 평가

In [ ]:
_, pred = evaluate(x_val_ts, y_val_ts, model, loss_fn, device)

In [ ]:
pred = np.where(pred.numpy() > .5, 1, 0)

* confusion matrix

In [ ]:
confusion_matrix(y_val_ts.numpy(), pred)

* classification_report

In [ ]:
print(classification_report(y_val_ts.numpy(), pred))

----

## **5.딥러닝3 : hidden layer**
* 이제 레이어를 추가해 보겠습니다.


### (1) 모델 선언

In [ ]:
n_feature = x.shape[1]

# 모델 구조 설계
model = nn.Sequential(
            nn.Linear(n_feature, 5),    # 은닉층
             ,                          # 은닉층의 활성화함수
            nn.Linear(5, 1),            # 출력층
                                        # 출력층의 활성화함수
        ).to(device)

print(model)

* Loss function과 Optimizer

In [ ]:
loss_fn =           # Bineary Cross Entropy
optimizer =

### (2) 학습

In [ ]:
epochs = 50
tr_loss_list, val_loss_list = [], []

for t in range( ):
    tr_loss =
    val_loss,_ =

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 학습된 파라미터 확인

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name}, Value: {param.data}")

* 학습 곡선

In [ ]:
dl_learning_curve(tr_loss_list, val_loss_list)

### (4) 모델 평가

In [ ]:
_, pred =

In [ ]:
pred =  (  > .5, 1, 0)

* confusion matrix

In [ ]:
confusion_matrix(y_val_ts.numpy(), pred)

* classification_report

In [ ]:
print(classification_report(y_val_ts.numpy(), pred))

## **6.실습**

### (1) 실습1

* 다음의 구조를 보고 모델을 설계하시오.

        Sequential(
        (0): Linear(in_features=8, out_features=10, bias=True)
        (1): ReLU()
        (2): Linear(in_features=10, out_features=5, bias=True)
        (3): ReLU()
        (4): Linear(in_features=5, out_features=1, bias=True)
        (5): Sigmoid()
        )

### (2) 실습2

* 이번에는 여러분이 원하는 대로 설계하고, 학습해 봅시다.
